In [ ]:
%matplotlib inline
import random
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from champ.adapters_cython import simple_hamming_distance
from champ import misc, hdf5tools
import tifffile
import os
import glob
import sys
import h5py
import numpy as np
import re

In [ ]:
date = ''
image_dir = '.'

In [ ]:
h5_fpaths = glob.glob(os.path.join(image_dir, '*.h5'))
for i, h5_fpath in enumerate(h5_fpaths):
    if 'phix' in h5_fpath.lower():
        h5_fpaths.pop(i)
h5_fpaths.sort(key=misc.parse_concentration)
h5_fpaths

In [ ]:
im_idx = 120
vmin, vmax = {}, {}
for h5_fpath in h5_fpaths:
    with h5py.File(h5_fpath) as f:
        g_keys = f.keys()
        g_keys.sort()
        
        for i, g_key in enumerate(g_keys):
            g = f[g_key]
            im_keys = g.keys()
            im_keys.sort()
            im_key = im_keys[im_idx]
            im = np.array(g[im_key])
            if g_key in vmin:
                vmin[g_key] = min(im.min(), vmin[g_key])
                vmax[g_key] = max(im.max(), vmax[g_key])
            else:
                vmin[g_key] = im.min()
                vmax[g_key] = im.max()

print vmin, vmax
for h5_fpath in h5_fpaths:
    with h5py.File(h5_fpath) as f:
        g_keys = f.keys()
        g_keys.sort()
        
        fig, axes = plt.subplots(1, len(g_keys), figsize=(8*len(g_keys), 8))
        try:
            list(axes)
        except:
            axes = [axes]
        for i, (ax, g_key) in enumerate(zip(axes, g_keys)):
            g = f[g_key]
            im_keys = g.keys()
            im_keys.sort()
            im_key = im_keys[im_idx]
            im = np.array(g[im_key])

            ax.matshow(im, vmin=vmin[g_key], vmax=vmax[g_key], cmap='viridis')
            ax.set_title('{:.0f}pM {} {}'.format(misc.parse_concentration(h5_fpath), g_key, im_key))
            ax.set_xticks([])
            ax.set_yticks([])
            
            rc = 150
            w = 75
            ax.plot([rc, rc, rc+w, rc+w, rc], [rc, rc+w, rc+w, rc, rc], 'w', linewidth=2)
            ax.plot([rc+w, im.shape[0]], [rc, im.shape[1]/2.0], 'w', linewidth=2)
            ax.plot([rc, im.shape[0]/2.0], [rc+w, im.shape[1]], 'w', linewidth=2)
            pos = ax.get_position()
            new_pos = [pos.x0 + pos.width/2.0, pos.y0, pos.width/2.0, pos.height/2.0]
            ax = fig.add_axes(new_pos)
            print(rc, rc+w)
            ax.matshow(im[rc:rc+w, rc:rc+w], vmin=vmin[g_key], vmax=vmax[g_key], cmap='viridis')
            ax.set_xticks([])
            ax.set_yticks([])

# Look for saturation

In [ ]:
for h5_fpath in h5_fpaths:
    tif_dir = os.path.splitext(h5_fpath)[0]
    tif_fpaths = glob.glob(os.path.join(tif_dir, '*.tif'))
    tif_fpath = tif_fpaths[0]

    fname = os.path.split(tif_fpath)[1]
    print(fname)
    m = fov_regex.search(fname)
    channel = m.group('channel')
    minor_axis_pos, Major_axis_pos = int(m.group('minor')), int(m.group('major'))

    with tifffile.TiffFile(tif_fpath) as tif:
        summary = tif.micromanager_metadata['summary']
        
        bit_depth = summary['BitDepth']
        sat_val = 2**bit_depth

        # Find channel names and assert unique 
        channel_names = [name.replace(' ', '_').replace('(', '').replace(')', '')
                         for name in summary['ChNames']]
        assert summary['Channels'] == len(channel_names) == len(set(channel_names)), channel_names

        # channel_idxs map tif pages to channels
        channel_idxs = tif.micromanager_metadata['index_map']['channel']

        # Setup defaultdict
        h, w = summary['Height'], summary['Width']
        def hw_zeros():
            return np.zeros((h, w), dtype=np.int)
        summed_images = defaultdict(hw_zeros)
        all_vals = defaultdict(list)

        # Add images and vals
        for channel_idx, page in zip(channel_idxs, tif.pages):
            all_vals[channel_idx].extend(page.asarray().flatten())
            summed_images[channel_idx] += page.asarray()
            
    nchannels = len(channel_names)
    fig, axes = plt.subplots(2, nchannels, figsize=(8*nchannels, 8))
    if len(axes.shape) == 1:
        axes0 = [axes[0]]
        axes1 = [axes[1]]
    else:
        assert len(axes.shape) == 2, axes.shape
        axes0 = axes[0, :]
        axes1 = axes[1, :]
    for ch_idx, (ax, ch_name) in enumerate(zip(axes0, channel_names)):
        ax.hist(all_vals[ch_idx], 100, histtype='step')
        ax.set_title('{} pM Channel {} Individual Images'.format(misc.parse_concentration(fname), ch_name))
        ax.plot([sat_val]*2, ax.get_ylim(), 'k--', alpha=0.5)
        ax.grid(False)
        ax.set_axis_bgcolor('white')
    for ch_idx, (ax, ch_name) in enumerate(zip(axes1, channel_names)):
        ax.hist(summed_images[ch_idx].flatten(), 100, color='darkgoldenrod', histtype='step')
        ax.set_title('{} pM Channel {} Summed Images'.format(misc.parse_concentration(fname), ch_name))
        ax.grid(False)
        ax.set_axis_bgcolor('white')


In [ ]:
from sklearn.neighbors import KernelDensity
from scipy.optimize import minimize

In [ ]:
def build_im_kdf(im, w=200):
    rmid, cmid = int(im.shape[0]/2), int(im.shape[1]/2)
    vmin, vmax = im.min(), im.max()
    bandwidth = (vmax - vmin)/100
    kdf = KernelDensity(bandwidth=bandwidth)
    hw = w/2
    pct95 = vmin + 0.95 * (vmax - vmin)
    vals = [v for v in im[rmid-hw:rmid+hw, cmid-hw:cmid+hw].flatten() if v < pct95]  # remove saturation
    kdf.fit(np.array(vals).reshape(len(vals), 1))
    return kdf

In [ ]:
def get_mode(im):
    kdf = build_im_kdf(im)
    def neg_kdf(x):
        return -kdf.score(x)
    res = minimize(neg_kdf, x0=np.median(im.flatten()), method='Nelder-Mead')
    assert res.success, res
    return res.x

In [ ]:
im_idx = 0
for h5_fpath in h5_fpaths[-1:]:
    with h5py.File(h5_fpath) as f:
        g_keys = f.keys()
        g_keys.sort()
        
        fig, axes = plt.subplots(1, len(g_keys), figsize=(8*len(g_keys), 8))
        try:
            list(axes)
        except:
            axes = [axes]
        for i, (ax, g_key) in enumerate(zip(axes, g_keys)):
            g = f[g_key]
            im_keys = g.keys()
            im_keys.sort()
            im_key = im_keys[im_idx]
            im = np.array(g[im_key])

            h, bin_edges, patches = ax.hist(im.flatten(), 100, normed=True)
            
            kdf = build_im_kdf(im)
            npoints = 1000
            x = np.linspace(bin_edges[0], bin_edges[-1], npoints).reshape(npoints, 1)
            y = np.exp(kdf.score_samples(x))
            print x.shape, y.shape, y.max()
            ax.plot(x.flatten(), y)
            
            mode = get_mode(im)
            ylim = ax.get_ylim()
            ax.plot([mode]*2, ylim, '--')
            ax.set_ylim(ylim)
            ax.set_xlim((0, 2*mode))
            
            ax.set_title('{:.0f}pM {} {}'.format(misc.parse_concentration(h5_fpath), g_key, im_key))


In [ ]:
im_idx = 0
for h5_fpath in h5_fpaths[-1:]:
    with h5py.File(h5_fpath) as f:
        g_keys = f.keys()
        g_keys.sort()
        for i, g_key in enumerate(g_keys):
            g = f[g_key]
            im_keys = g.keys()
            im_keys.sort()
            im_key = im_keys[im_idx]
            im = np.array(g[im_key])
            
            print get_mode(im)